In [1]:
# change directory location if needed:    
data_dir = "/scratch/new_courier_pilot/" ## pilot data directory
# data_dir = "/scratch/EFRCourier_reports/" ## hospital data directory
time_window=10000 # you can chose how long do you want to visualize the data
exp = 'EFRCourierOpenLoop'
subject = 'R1620J' 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import glob
import cmlreaders as cml
import json
from matplotlib.ticker import FuncFormatter
import warnings
import numpy as np
import pandas as pd

# Get the full data index
whole_df = cml.CMLReader.get_data_index()

# Define the experiment
exp = 'EFRCourierOpenLoop'

# Get unique subjects for the experiment
subjects = whole_df.query('experiment == @exp')['subject'].unique()

# Initialize results storage
results = []

# Loop through each subject
for subject in subjects:
    print(f"Processing subject: {subject}")
    
    # Get subject data
    sub_df = whole_df.query('experiment == @exp and subject == @subject')
    
    # Get all sessions for this subject
    sessions = sub_df['session'].unique()
    
    # Loop through each session
    for session in sessions:
        print(f"  Processing session: {session}")
        
        try:
            # Create reader for this subject and session
            reader = cml.CMLReader(subject, exp, session=session)
            evs = reader.load('task_events')
            
            # Add the inside_stimuli variable
            evs['inside_stimuli'] = -999
            stimuli_indices = evs[evs['type'] == 'STIM'].index
            for i in stimuli_indices:
                current_offset = evs.loc[i, 'eegoffset']
                
                # Keep checking subsequent rows until difference >= 3000
                j = i + 1
                rows_to_assign = []  # Store indices of rows to assign
                
                while j < len(evs):
                    next_offset = evs.loc[j, 'eegoffset']
                    time_diff = abs(next_offset - current_offset)
                    
                    if time_diff < 3000:
                        # Still within 3 seconds, add this row to assignment list
                        rows_to_assign.append(j)
                        j += 1
                    else:
                        # Found a row with difference >= 4000, stop checking
                        break
                
                # Assign burst_freq to all rows that were within 4000ms of the STIM
                for row_idx in rows_to_assign:
                    evs.loc[row_idx, 'inside_stimuli'] = evs.loc[i, 'stim_params']['burst_freq']
            
            # Filter for intrusions (intrusion == 1)
            
            evs = add_phase(evs)
            intrusion_events = evs[(evs['intrusion'] > 0) | (evs['intrusion'] == -1)]
            none_intrusion_events = evs[evs['intrusion'] == 0]

                # Count intrusions for each stimulation condition
            intrusion_count_3hz = (intrusion_events['inside_stimuli'] == 3).sum()
            intrusion_count_8hz = (intrusion_events['inside_stimuli'] == 8).sum()
            intrusion_count_no_stim = (intrusion_events['inside_stimuli'] == -999).sum()
            intrusion_count_total = len(intrusion_events)

                # Count none intrusions for each stimulation condition
            none_intrusion_count_3hz = (none_intrusion_events['inside_stimuli'] == 3).sum()
            none_intrusion_count_8hz = (none_intrusion_events['inside_stimuli'] == 8).sum()
            none_intrusion_count_no_stim = (none_intrusion_events['inside_stimuli'] == -999).sum()
            none_intrusion_count_total = len(none_intrusion_events)

                # Store results
            
            session_result = {
                        'subject': subject,
                        'session': session,
                        'intrusion_count_3hz': intrusion_count_3hz,
                        'intrusion_count_8hz': intrusion_count_8hz,
                        'intrusion_count_no_stim': intrusion_count_no_stim,
                        'intrusion_count_total': intrusion_count_total,
                        'none_intrusion_count_3hz': none_intrusion_count_3hz,
                        'none_intrusion_count_8hz': none_intrusion_count_8hz,
                        'none_intrusion_count_no_stim': none_intrusion_count_no_stim,
                        'none_intrusion_count_total': none_intrusion_count_total,
                        'relative_rate_3hz': intrusion_count_3hz / none_intrusion_count_3hz if none_intrusion_count_3hz != 0 else None,
                        'relative_rate_8hz': intrusion_count_8hz / none_intrusion_count_8hz if none_intrusion_count_8hz != 0 else None,
                        'relative_rate_no_stim': intrusion_count_no_stim / none_intrusion_count_no_stim if none_intrusion_count_no_stim != 0 else None
                }
            results.append(session_result)
            
            print(f"    Intrusions - 3Hz: {intrusion_count_3hz}, 8Hz: {intrusion_count_8hz}, No stim: {intrusion_count_no_stim}, Total: {intrusion_count_total}")
            
        except Exception as e:
            print(f"    Error processing session {session}: {str(e)}")
            continue

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display summary statistics
print("\n" + "="*50)
print("INTRUSION ANALYSIS SUMMARY")
print("="*50)

if len(results_df) > 0:
    print(f"Total subjects processed: {results_df['subject'].nunique()}")
    print(f"Total sessions processed: {len(results_df)}")
    print(f"\nIntrusion statistics across all sessions:")
    print(f"Mean intrusions per session:")
    print(f"  3Hz stimulation: {results_df['intrusion_count_3hz'].mean():.3f} ± {results_df['intrusion_count_3hz'].std():.3f}")
    print(f"  8Hz stimulation: {results_df['intrusion_count_8hz'].mean():.3f} ± {results_df['intrusion_count_8hz'].std():.3f}")
    print(f"  No stimulation: {results_df['intrusion_count_no_stim'].mean():.3f} ± {results_df['intrusion_count_no_stim'].std():.3f}")
    print(f"  Total: {results_df['intrusion_count_total'].mean():.3f} ± {results_df['intrusion_count_total'].std():.3f}")
    print(f"\nTotal intrusions across all sessions:")
    print(f"  3Hz stimulation: {results_df['intrusion_count_3hz'].sum()}")
    print(f"  8Hz stimulation: {results_df['intrusion_count_8hz'].sum()}")
    print(f"  No stimulation: {results_df['intrusion_count_no_stim'].sum()}")
    print(f"  Total: {results_df['intrusion_count_total'].sum()}")
    
    # Display the results DataFrame
    print(f"\nDetailed results:")
    print(results_df.to_string(index=False))
else:
    print("No results to display. Check for errors in processing.")
   

Processing subject: FBG490
  Processing session: 0
    Error processing session 0: name 'add_phase' is not defined
  Processing session: 1
    Error processing session 1: name 'add_phase' is not defined
  Processing session: 2
    Error processing session 2: name 'add_phase' is not defined
Processing subject: FBG491
  Processing session: 1
    Error processing session 1: name 'add_phase' is not defined
  Processing session: 2
    Error processing session 2: name 'add_phase' is not defined
Processing subject: FR491
  Processing session: 1
    Error processing session 1: Unable to find the requested file in any of the expected locations:
 /protocols/pyfr/subjects/FR491/experiments/EFRCourierOpenLoop/sessions/1/behavioral/current_processed/task_events.json
/data/events/pyFR/FR491_None_events.mat
  Processing session: 2
    Error processing session 2: Unable to find the requested file in any of the expected locations:
 /protocols/pyfr/subjects/FR491/experiments/EFRCourierOpenLoop/sessions/

In [2]:
evs[evs['type']=='REC_START']
evs['trial'].unique()

array([-999,   -1,    0,    1,    2,    3])

In [3]:
intrusion_events[['intruded','intrusion','inside']]

NameError: name 'intrusion_events' is not defined

In [ ]:
evs["type"].unique()

In [4]:
evs["phase"]=1
new_evs = pd.DataFrame()  # Initialize as DataFrame, not list
for ind, list_evs in evs.groupby('trial'):  # 'trial' is the equivalent of a delivery day
    list_evs = list_evs.copy()  # Avoid SettingWithCopyWarning
    if list_evs['trial'].unique()[0] != -999:
        list_evs['phase'] = 'nan'
        if not list_evs[list_evs['type'] == 'TRIAL_START'].index.empty and not list_evs[list_evs['type'] == 'TRIAL_END'].index.empty:
            # find indexes of encoding start and end
            rec_start_indices = list_evs[list_evs['type'] == 'REC_START'].index
            if not rec_start_indices.empty:
                rec_start_index = rec_start_indices[0]
                start_time = list_evs.loc[rec_start_index]['eegoffset']
                # Find the actual start index based on time condition
                i = 0
                rec_start_index_found = True
                actual_start_index = rec_start_index  # Initialize with default
                
                while rec_start_index_found:
                    current_idx = rec_start_index + i
                    print(i)
                    # Check bounds for iloc access
                    check_time = evs.iloc[current_idx]['eegoffset']  # Use relative index i
                    if check_time - start_time < 3000:
                        i += 1
                        print(check_time - start_time)
                    else:
                        rec_start_index_found = False
                        actual_start_index = rec_start_index+i
                        print(i)# Get actual index from list_evs
                rec_end_indices = list_evs[list_evs['type'] == 'REC_STOP'].index
                if not rec_end_indices.empty:
                    rec_end_index = rec_end_indices[0]
                    # Now assign to original evs DataFrame using the original indices
                    evs.loc[actual_start_index:rec_end_index, 'phase'] = 'retrieval'
                    print('get one value', actual_start_index,rec_end_index)


0
0
1
387
2
2434
3
2805
4
4
get one value 110 140
0
0
1
540
2
1540
3
2523
4
2797
5
5
get one value 200 219
0
0
1
257
2
967
3
1902
4
2804
5
2902
6
6
get one value 276 296
0
0
1
142
2
1082
3
2507
4
2797
5
5
get one value 353 379


In [ ]:
check_time = evs.iloc[current_idx]['eegoffset']

In [ ]:
current_idx


In [ ]:
rec_start_indices = list_evs[list_evs['type'] == 'REC_START'].index
rec_start_indices

In [5]:

pd.set_option('display.max_rows', None)
evs[['phase','type']]

,phase,type
0,1,START
1,1,store mappings
2,1,TL_START
3,1,pointing begins
4,1,pointing finished
5,1,pointing begins
6,1,pointing finished
7,1,POINTER_ON
8,1,pointing begins
9,1,pointing finished


In [ ]:
list_evs[list_evs['type'] == 'REC_START']

In [ ]:
list_evs.loc[start_index]

In [ ]:
actual_start_index

In [ ]:
rec_start_index = list_evs[list_evs['type']=='REC_START'].index[0]
rec_end_index = list_evs[list_evs['type']=='REC_STOP'].index[0]
list_evs.loc[rec_start_index:]['phase'] = 'retrieval'

In [ ]:
list_evs.loc[301]

In [ ]:
# change directory location if needed:    
data_dir = "/scratch/new_courier_pilot/" ## pilot data directory
# data_dir = "/scratch/EFRCourier_reports/" ## hospital data directory
time_window=10000 # you can chose how long do you want to visualize the data
exp = 'EFRCourierOpenLoop'
subject = 'R1620J' 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import glob
import cmlreaders as cml
import json
from matplotlib.ticker import FuncFormatter
import warnings
import numpy as np
import pandas as pd

# Get the full data index
whole_df = cml.CMLReader.get_data_index()

# Define the experiment
exp = 'EFRCourierOpenLoop'

# Get unique subjects for the experiment
subjects = whole_df.query('experiment == @exp')['subject'].unique()

# Initialize results storage
results = []

# Loop through each subject
for subject in subjects:
    print(f"Processing subject: {subject}")
    
    # Get subject data
    sub_df = whole_df.query('experiment == @exp and subject == @subject')
    
    # Get all sessions for this subject
    sessions = sub_df['session'].unique()
    
    # Loop through each session
    for session in sessions:
        print(f"  Processing session: {session}")
        
        try:
            # Create reader for this subject and session
            reader = cml.CMLReader(subject, exp, session=session)
            evs = reader.load('task_events')
          

In [ ]:
evs['phase']=1